In [6]:

import torch
import torchvision.transforms
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader
import time



# 定义训练设备
# 利用GPU为cuda
device = torch.device('cpu')

# 下载并加载 CIFAR-10 训练数据集
# root 指定数据集存储的根目录；train=True 表示加载训练集；
# transform 将数据转换为 Tensor 类型；download=True 表示如果数据集不存在则进行下载
train_data = torchvision.datasets.CIFAR10(root= r'D:\Desktop\数据集', train=True, transform=torchvision.transforms.ToTensor(), download=True)
# 下载并加载 CIFAR-10 测试数据集
test_data = torchvision.datasets.CIFAR10(root= r'D:\Desktop\数据集', train=False, transform=torchvision.transforms.ToTensor(), download=True)

# 计算训练集和测试集的长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"训练数据集的长度为: {train_data_size}")
print(f"测试数据集的长度为: {test_data_size}")

# 加载数据
# 使用 DataLoader 对训练数据进行批量加载，batch_size=64 表示每个批次包含 64 个样本
train_dataloader = DataLoader(train_data, batch_size=64)
# 对测试数据进行批量加载
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型
class zzy(nn.Module):
    def __init__(self):
        super(zzy, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64,10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 实例化自定义的网络模型 zzy（假设在 model.py 中定义）
zzy1 = zzy()


zzy1 = zzy1.to(device)
# 定义损失函数
# 使用交叉熵损失函数，常用于多分类问题
loss_fn = nn.CrossEntropyLoss()

loss_fn = loss_fn.to(device)

# 优化器创建
# 学习率设置为 0.01
learning_rate = 1e-2
# 使用随机梯度下降（SGD）优化器，对 zzy1 模型的参数进行优化
optimier = torch.optim.SGD(zzy1.parameters(), lr=learning_rate)

# 设置训练网络参数
# 记录总的训练步数
total_train_step = 0
# 记录总的测试步数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加 TensorBoard 用于可视化训练过程
writer = SummaryWriter('log_dir')
# 开始训练循环，共训练 epoch 轮
start_time = time.time()
for i in range(epoch):
    print(f'--------第{i + 1}次训练--------')
    # 遍历训练数据加载器中的每个批次
    for data in train_dataloader:
        # 从数据批次中解包图像和对应的标签
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        # 将图像输入到模型中进行前向传播，得到模型的输出
        outputs = zzy1(imgs)
        # 计算模型输出与真实标签之间的损失
        loss = loss_fn(outputs, targets)

        # 梯度清零，防止梯度累积
        optimier.zero_grad()
        # 反向传播，计算梯度
        loss.backward()
        # 根据计算得到的梯度更新模型的参数
        optimier.step()

        # 训练步数加 1
        total_train_step += 1
        # 每训练 100 步，打印一次训练信息并将训练损失写入 TensorBoard
        if total_train_step % 100 == 0:
            end_time =time.time()
            print(f'用时为{end_time-start_time}')
            print(f'训练次数：{total_train_step}，Loss:{loss.item()}')
            # 将训练损失添加到 TensorBoard 中，用于后续可视化
            writer.add_scalar('train_loss', loss.item(), total_train_step)

    # 测试步骤开始
    # 初始化总的测试损失为 0
    total_test_loss = 0
    # 上下文管理器，在测试过程中不进行梯度计算，减少内存消耗
    with torch.no_grad():
        # 遍历测试数据加载器中的每个批次
        for data in test_dataloader:
            # 从数据批次中解包图像和对应的标签
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            # 将图像输入到模型中进行前向传播，得到模型的输出
            outputs = zzy1(imgs)
            # 计算模型输出与真实标签之间的损失
            loss = loss_fn(outputs, targets)
            # 累加测试损失
            total_test_loss += loss.item()
    # 这里存在错误，应该打印整体的测试损失，而不是 total_test_step
    print(f'整体的测试损失: {total_test_loss}')
    # 将测试损失添加到 TensorBoard 中，用于后续可视化
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    # 测试步数加 1
    total_test_step += 1

# 关闭 SummaryWriter，释放资源
writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为: 50000
测试数据集的长度为: 10000
--------第1次训练--------
用时为2.803363800048828
训练次数：100，Loss:2.2882862091064453
用时为5.325129985809326
训练次数：200，Loss:2.2752037048339844
用时为7.95932936668396
训练次数：300，Loss:2.222085952758789
用时为10.740933179855347
训练次数：400，Loss:2.107771873474121
用时为13.729658365249634
训练次数：500，Loss:2.0390682220458984
用时为16.50631070137024
训练次数：600，Loss:1.9945814609527588
用时为19.146372079849243
训练次数：700，Loss:2.007131576538086
整体的测试损失: 315.87202501296997
--------第2次训练--------
用时为24.215763092041016
训练次数：800，Loss:1.85383939743042
用时为26.893329620361328
训练次数：900，Loss:1.8599872589111328
用时为29.93969178199768
训练次数：1000，Loss:1.9169049263000488
用时为32.71237516403198
训练次数：1100，Loss:1.961837649345398
用时为35.35830593109131
训练次数：1200，Loss:1.7128417491912842
用时为38.14988422393799
训练次数：1300，Loss:1.6631531715393066
用时为40.86748766899109
训练次数：1400，Loss:1.7120702266693115
用时为43.8605637550354
训练次数：1500，Loss:1.803911805152893
整体的测试损